In [2]:
import matplotlib.pyplot as plt
import time
import tensorflow as tf

from tensorflow.keras import models, layers
from tensorflow.keras.datasets import cifar10


(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32') / 255.0

# Flatten labels
y_train = y_train.flatten()
y_test  = y_test.flatten()

# Residual Block
def residual_block(input_tensor, filters, stride=1):
    x = layers.Conv2D(filters, (3, 3), strides=stride, padding="same", activation="relu")(input_tensor)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(filters, (3, 3), padding="same")(x)
    x = layers.BatchNormalization()(x)

    shortcut = input_tensor
    if stride != 1 or input_tensor.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, (1, 1), strides=stride, padding="same")(input_tensor)

    x = layers.add([x, shortcut])
    x = layers.Activation("relu")(x)
    return x

# Build ResNet-10
def build_resnet10(input_shape=(32, 32, 3), num_classes=10):

    inputs = layers.Input(shape=input_shape)

    # Initial conv
    x = layers.Conv2D(64, (3, 3), strides=1, padding="same", activation="relu")(inputs)
    x = layers.BatchNormalization()(x)

    # 5 blocks of 64 filters
    for _ in range(5):
        x = residual_block(x, 64)

    # 5 blocks of 128 filters (downsample on first)
    for i in range(5):
        x = residual_block(x, 128, stride=2 if i == 0 else 1)

    # Global pool & classifier
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    return models.Model(inputs, outputs)


resnet10 = build_resnet10()
resnet10.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

print(resnet10.summary())

class TenEpochLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1:3d} | "
                  f"loss={logs['loss']:.4f} | acc={logs['accuracy']:.4f} | "
                  f"val_loss={logs['val_loss']:.4f} | val_acc={logs['val_accuracy']:.4f}")

# Train 300 epochs
start_time = time.time()

history = resnet10.fit(
    x_train, y_train,
    epochs=300,
    validation_data=(x_test, y_test),
    callbacks=[TenEpochLogger()],
    verbose=0
)

training_time = time.time() - start_time

# Final evaluation
final_loss, final_accuracy = resnet10.evaluate(x_test, y_test, verbose=0)

print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Final Training Loss: {history.history['loss'][-1]:.4f}")
print(f"Final Validation Accuracy: {final_accuracy:.4f}")

# Plot Accuracy Graph
plt.plot(history.history["accuracy"], label="Training Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("ResNet-10 Training vs Validation Accuracy (300 Epochs)")
plt.legend()
plt.grid(True)
plt.show()

# Plot Loss Graph
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("ResNet-10 Training vs Validation Loss (300 Epochs)")
plt.legend()
plt.grid(True)
plt.show()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_31 (Conv2D)  │ (None, 32, 32,    │      1,792 │ input_layer_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 32, 32,    │        256 │ conv2d_31[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_32 (Conv2D)  │ (None, 32, 32,    │     36,928 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        256 │ conv2d_32[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_33 (Conv2D)  │ (None, 32, 32,    │     36,928 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        256 │ conv2d_33[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_14 (Add)        │ (None, 32, 32,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 32, 32,    │          0 │ add_14[0][0]      │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_34 (Conv2D)  │ (None, 32, 32,    │     36,928 │ activation[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        256 │ conv2d_34[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_35 (Conv2D)  │ (None, 32, 32,    │     36,928 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        256 │ conv2d_35[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_15 (Add)        │ (None, 32, 32,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 32, 32,    │          0 │ add_15[0][0]      │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_36 (Conv2D)  │ (None, 32, 32,    │     36,928 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        256 │ conv2d_36[0][0] 

 Total params: 1,790,730 (6.83 MB)

 Trainable params: 1,786,762 (6.82 MB)

 Non-trainable params: 3,968 (15.50 KB)

None


KeyboardInterrupt: 

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time

from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = y_train.flatten(), y_test.flatten()



# Residual Block (supports weight decay, dropout, batchnorm)
def residual_block(x, filters, stride=1, weight_decay=None, dropout_rate=None, use_bn=True):

    reg = regularizers.l2(weight_decay) if weight_decay else None
    shortcut = x

    # Conv 1
    x = layers.Conv2D(filters, 3, strides=stride, padding="same",
                      kernel_regularizer=reg)(x)
    if use_bn:
        x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Optional Dropout
    if dropout_rate:
        x = layers.Dropout(dropout_rate)(x)

    # Conv 2
    x = layers.Conv2D(filters, 3, padding="same",
                      kernel_regularizer=reg)(x)
    if use_bn:
        x = layers.BatchNormalization()(x)

    # Match shortcut size
    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, 1, strides=stride,
                                 kernel_regularizer=reg)(shortcut)
        if use_bn:
            shortcut = layers.BatchNormalization()(shortcut)

    x = layers.add([x, shortcut])
    return layers.ReLU()(x)



# Build ResNet-10 with optional regularization
def build_resnet10(weight_decay=None, dropout_rate=None, use_bn=True):
    inputs = layers.Input(shape=(32, 32, 3))

    x = layers.Conv2D(64, 3, padding="same", activation="relu")(inputs)
    if use_bn:
        x = layers.BatchNormalization()(x)

    # 5 residual blocks at 64 filters
    for _ in range(5):
        x = residual_block(x, 64,
                           weight_decay=weight_decay,
                           dropout_rate=dropout_rate,
                           use_bn=use_bn)

    # 5 residual blocks at 128 filters
    for i in range(5):
        x = residual_block(x, 128,
                           stride=2 if i == 0 else 1,
                           weight_decay=weight_decay,
                           dropout_rate=dropout_rate,
                           use_bn=use_bn)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(10, activation="softmax")(x)

    return models.Model(inputs, outputs)

class SimpleLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch+1:02d} | acc={logs['accuracy']:.4f} | val_acc={logs['val_accuracy']:.4f}")

# Training function
def run_experiment(name, weight_decay=None, dropout_rate=None, use_bn=True):

    print(f"\n==================== {name} ====================\n")

    model = build_resnet10(
        weight_decay=weight_decay,
        dropout_rate=dropout_rate,
        use_bn=use_bn
    )

    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])

    start = time.time()
    history = model.fit(
        x_train, y_train,
        epochs=30,
        validation_data=(x_test, y_test),
        callbacks=[SimpleLogger()],
        verbose=0
    )
    elapsed = time.time() - start

    final_val_acc = history.history["val_accuracy"][-1]

    print(f"\n{name} — Final Validation Accuracy: {final_val_acc:.4f}")
    print(f"{name} — Training Time: {elapsed:.2f} seconds")

    return history

# Weight Decay λ = 0.001
history_wd = run_experiment(
    "Weight Decay (lambda = 0.001)",
    weight_decay=0.001,
    dropout_rate=None,
    use_bn=True
)

# Dropout p = 0.3
history_dropout = run_experiment(
    "Dropout (p = 0.3)",
    weight_decay=None,
    dropout_rate=0.3,
    use_bn=True
)

# Batch Normalization Only
history_bn = run_experiment(
    "Batch Normalization Only",
    weight_decay=None,
    dropout_rate=None,
    use_bn=True
)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step

==================== Weight Decay (lambda = 0.001) ====================



KeyboardInterrupt: 